In [1]:
from src.run_model import PEFTModel
from src.peft_search_space import PEFTSearchSpace
from src.dataset_wrapper import PEFTDataset
from pruning_methods import prune_model
from utils.gpu_memory_plot import get_free_gpu_memory

import argparse
import logging
import time
from copy import deepcopy
import os
import torch

logger = logging.getLogger('controller')
logger.setLevel(logging.INFO)  # 设置日志级别
time_str = time.strftime('%Y-%m-%d_%H:%M:%S', time.localtime())
day_str = time.strftime('%Y-%m-%d', time.localtime())
output_dir = f'outputs/{day_str}'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
file_handler = logging.FileHandler(
    f'outputs/{day_str}/output_{time_str}.log', mode='w')
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

logger.info('Start loading dataset')
dataset = PEFTDataset(
    'glue', 'cola', test_size=100, train_size=300).get_dataset()

parser = argparse.ArgumentParser()
parser.add_argument('--lora', type=int, nargs='+')
parser.add_argument('--adapter', type=int, nargs='+')
parser.add_argument('--base_lora', type=int)
parser.add_argument('--base_adapter', type=int)
args = parser.parse_args(args=[])

searched_args = []
searched_res = []

# baseline
logger.info('Start baseline')


def wrapper(search_list, idt='lora'):
    global model, res, configs, gradients, activations
    args = parser.parse_args(args=[])
    if idt == 'lora':
        args.lora = search_list
    elif idt == 'adapter':
        args.adapter = search_list
    args.epochs = 1
    args.instructs = 1
    configs = PEFTSearchSpace(args).get_config()
    model = PEFTModel(configs, dataset).half()
    res, gradients, activations = model.run()
    logger.info(f'Result {res} for {search_list}')
    model = None
    gradients = None
    activations = None
    torch.cuda.empty_cache()


def wrapper2(search_list, search_list2):
    global model, res, configs, gradients, activations
    args = parser.parse_args(args=[])
    args.lora = search_list
    args.adapter = search_list2
    args.epochs = 1
    args.instructs = 1
    configs = PEFTSearchSpace(args).get_config()
    model = PEFTModel(configs, dataset)
    res, gradients, activations = model.run()
    logger.info(f'Result for {search_list, search_list2}: {res}')


# logger.info("Start adapter baseline")
# wrapper([32] * 16, 'adapter')

# logger.info('Start lora baseline')
# wrapper([32] * 32, 'lora')

prune_turn = 10
logger.info('-----Start gradient test------')

origin_search_list = [32] * 32
origin_search_list2 = [32] * 32
search_list = deepcopy(origin_search_list)
search_list2 = deepcopy(origin_search_list2)

for _ in range(1):
    model = None
    torch.cuda.empty_cache()
    logger.info(
        f'Start searching for lora:{search_list} adapter:{search_list2}')
    args.lora = search_list
    args.adapter = search_list2
    args.epochs = 1
    configs = PEFTSearchSpace(args).get_config()
    model = PEFTModel(configs, dataset).half()
    res, gradients, activations = model.run()



/home/autopeft/anaconda3/envs/autopeft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/autopeft/anaconda3/envs/autopeft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.15s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


number of label classes: 2


/home/autopeft/anaconda3/envs/autopeft/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 300
}) Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 100
})
finished init


Training epoch 1: 100%|██████████| 19/19 [00:48<00:00,  2.58s/batch]


Epoch 2 || Validation loss: nan || Validation accuracy: 0.360000
{'eval_loss': nan, 'eval_accuracy': 0.36}


In [ ]:
import datasets


dataset_train=datasets.load_dataset('rotten_tomatoes',split="train[:100]")
dataset_test=datasets.load_dataset('rotten_tomatoes',split="test[:50%]")
dataset=datasets.DatasetDict({'train':dataset_train,'test':dataset_test})

def add_prefix(example):
    example["text"] = 'This is an prefix' + example["text"]
    return example

instructed_dataset=dataset.map(add_prefix)

print(instructed_dataset)
instructed_dataset['train']['text'][:10]


In [ ]:
from pruning_methods import prune_model
idx, idt = prune_model(
    model.model,
    task_name='my_module',
    opts=['lora', 'adapter'],
    p_method='gradient',
    top_p=12,
    print_names=True,
    gradients=gradients)
logger.info(f'Pruned layer: {idx, idt}')
search_list[int(idx)] = 0

In [7]:
params_need_record = [
    param for name, param in model.model.named_parameters()
    if param.requires_grad
]
print(len(params_need_record))
print(len(list(model.model.parameters())))

132
422


In [6]:
type(activations['module.model.layers.0.self_attn.q_proj.loras.my_module.lora_A'])

tuple